In [1]:
! pip install pymongo


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from mongo import MongoDBManager
import os
from dotenv import load_dotenv
import pandas as pd
import matplotlib.pyplot as plt
import re

# Retrieving data

In [4]:
load_dotenv()
uri = os.getenv('MONGODB_URI')
database_name = os.getenv('DATABASE_NAME')
mongo_manager = MongoDBManager(uri, database_name)

In [5]:
documents = mongo_manager.read_all_documents_in_collection_with_GridFS()

Erreur lors de la lecture du document ID 6706b7711ef76235b2459476: ac-yfls4dx-shard-00-02.odkctaa.mongodb.net:27017: connection closed (configured timeouts: connectTimeoutMS: 20000.0ms)


In [6]:
def data_recurcive_length(data):
    total = 0
    try:
        for key in data.keys():
            for profession in data[key].keys():
                for language in data[key][profession].keys():
                    print(f"{key} - {profession} - {language} : {len(data[key][profession][language])}")
                    total += len(data[key][profession][language])
        print(f"Total: {total}")
    except Exception as e:
        print(e)
        print(data[key][profession])

In [7]:
data_recurcive_length(documents)

US - Accompagnatrice spirituelle ou accompagnateur spirituel - en : 68
US - Accompagnatrice spirituelle ou accompagnateur spirituel - fr : 18
US - Accompagnatrice, accompagnateur, musicienne ou musicien dans de petits ensembles - fr : 27
US - Accompagnatrice, accompagnateur, musicienne ou musicien dans de petits ensembles - en : 2723
US - Actuaire - en : 108
US - Adjointe ou adjoint à la recherche - en : 236
US - Administratrice ou administrateur - fr : 8
US - Administratrice ou administrateur - en : 1678
US - Administratrice ou administrateur de contrats - fr : 25
US - Administratrice ou administrateur de contrats - en : 522
US - Administratrice ou administrateur de programmes de développement durable - fr : 12
US - Administratrice ou administrateur de programmes de développement durable - en : 1024
US - Administratrice ou administrateur de prêts commerciaux - fr : 51
US - Administratrice ou administrateur de prêts commerciaux - en : 999
US - Agent ou agente d’approvisionnement et de 

# Formating Datas

In [8]:
def data_to_sql_format(data):
    data_formated = []
    for country in documents.keys():
        for profession in documents[country].keys():
            for language in documents[country][profession].keys():
                    for data in documents[country][profession][language]:
                        data['country'] = country
                        data['profession'] = profession
                        data['language'] = language
                        data_formated.append(data)
    return data_formated                    
data_formated=data_to_sql_format(documents)
print(len(data_formated))

21457


In [9]:
import dedupe

def clear_data(data):
    df=pd.DataFrame(data)
    df = df.applymap(lambda x: tuple(x) if isinstance(x, list) else x)
    df=df.drop_duplicates()
    df.describe()
    countries = df["country"].drop_duplicates().to_list()
    professions = df["profession"].drop_duplicates().to_list()
    languages = df["language"].drop_duplicates().to_list()
    clear_data={}
    for country in countries:
        for profession in professions:
            for lang in languages:
                try:
                    if country not in clear_data:
                        clear_data[country] = {}
                    if profession not in clear_data[country]:
                        clear_data[country][profession] = {}
                    
                    filtered_df = df[(df["language"] == lang) & (df["profession"] == profession) & (df["country"] == country)]
                    clear_data[country][profession][lang] = filtered_df.to_dict('records')
                except Exception as e:
                    print(e)
                
    return clear_data

data_cleared = clear_data(data_formated)

C:\Users\PC\AppData\Local\Temp\ipykernel_12440\1776475680.py:5: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: tuple(x) if isinstance(x, list) else x)


In [10]:
data_recurcive_length(data_cleared)

US - Accompagnatrice spirituelle ou accompagnateur spirituel - en : 68
US - Accompagnatrice spirituelle ou accompagnateur spirituel - fr : 10
US - Accompagnatrice, accompagnateur, musicienne ou musicien dans de petits ensembles - en : 2723
US - Accompagnatrice, accompagnateur, musicienne ou musicien dans de petits ensembles - fr : 27
US - Actuaire - en : 84
US - Actuaire - fr : 0
US - Adjointe ou adjoint à la recherche - en : 236
US - Adjointe ou adjoint à la recherche - fr : 0
US - Administratrice ou administrateur - en : 1678
US - Administratrice ou administrateur - fr : 8
US - Administratrice ou administrateur de contrats - en : 409
US - Administratrice ou administrateur de contrats - fr : 21
US - Administratrice ou administrateur de programmes de développement durable - en : 1024
US - Administratrice ou administrateur de programmes de développement durable - fr : 12
US - Administratrice ou administrateur de prêts commerciaux - en : 999
US - Administratrice ou administrateur de prêt

# Visualizing Data

In [11]:
df = pd.DataFrame(data_formated)
df.describe()

,positionTitle,companyName,location,salaryInfoAndJobType,description,datePosted,salary,jobType,shiftAndSchedule,jobBenefits,updatedDate,country,profession,language
count,21457,21457,21457,21457,21457,21104,21457,21457,21457,21457,21457,21457,21457,21457
unique,8899,5077,4827,3325,7369,61,1944,174,328,2040,13,2,14,2
top,Office Administrator,Accenture Federal Services,Paris (75),Full-time,,,,[],,[],2024-09-13,US,"Agente commerciale ou agent commercial, politi...",en
freq,255,244,593,2850,193,4487,10034,6900,17231,10225,4490,14780,4228,15709


In [12]:
df[df['shiftAndSchedule'] != ''].describe()

,positionTitle,companyName,location,salaryInfoAndJobType,description,datePosted,salary,jobType,shiftAndSchedule,jobBenefits,updatedDate,country,profession,language
count,4226,4226,4226,4226,4226,4225,4226,4226,4226,4226,4226,4226,4226,4226
unique,2275,1611,1627,1179,1888,55,729,54,327,851,8,1,12,2
top,Office Administrator,"California State University, Monterey Bay",Remote,Full-time,Academic and Student Success Advisor\n\nJob no...,,,,[Monday to Friday],[],2024-09-13,US,"Accompagnatrice, accompagnateur, musicienne ou...",en
freq,118,63,100,570,29,1024,1464,1144,822,1330,1418,4226,886,4149


In [13]:
df[:5]

,positionTitle,companyName,location,salaryInfoAndJobType,description,datePosted,salary,jobType,shiftAndSchedule,jobBenefits,updatedDate,country,profession,language
0,Overnight Cabin Supervisor-SFO Airport,G2 Secure Staff,"780 South Airport Boulevard, San Francisco, CA...",$31 an hour - Full-time,G2 Secure Staff is expanding and looking to ad...,2024-09-06,,"[an hour , Full]",,"[401(k), 401(k) matching, Dental insurance, Em...",2024-09-09,US,Accompagnatrice spirituelle ou accompagnateur ...,en
1,Chaplain - N/E - New River Valley Medical Center,Carilion Clinic,"2900 Lamb Circle, Christiansburg, VA 24073",,Provides a ministry of spiritual care and emot...,2024-09-09,,[],"[Weekends as needed, Night shift, On call]",[],2024-09-11,US,Accompagnatrice spirituelle ou accompagnateur ...,en
2,Bethany Christian Church Residency - Washingto...,Leadership Pathway,"Washington, IN 47501","$1,000 a month - Full-time",Bethany Christian Church is a growing multi-si...,2024-08-10,"$1,000 a month","[an hour , Full]",,[],2024-09-09,US,Accompagnatrice spirituelle ou accompagnateur ...,en
3,Groups Director/Pastor (Plainfield Campus),Traders Point Christian Church Inc,"2002 Stafford Road, Plainfield, IN 46168",Full-time,CULTURE & SPIRITUAL LEADERSHIP\n\nTraders Poin...,2024-09-09,,[],,[Leadership training provided],2024-09-11,US,Accompagnatrice spirituelle ou accompagnateur ...,en
4,Personal Care Attendant,EveryAge,"Lexington, NC 27292",Full-time,Carolina SeniorCare is a non-profit provider o...,2024-09-09,,[],,[],2024-09-11,US,Accompagnatrice spirituelle ou accompagnateur ...,en


In [28]:
df = df.applymap(lambda x: tuple(x) if isinstance(x, list) else x)
df = df.drop_duplicates()

df = df[((df['positionTitle'] != '') & (df['description'] != '')) | df['positionTitle'] != '']
df["jobType"] = df["jobType"].replace(' () ', '')
"""
df['jobType'] = df.apply(
    lambda x: tuple(list(x['jobType']) + list(x['salary'])) 
    if not re.search(r'\d+', str(x['salary'])) 
    else x['jobType'], 
    axis=1
)
"""
"""
df['salary'] = df['salary'].map(
    lambda x: '' 
    if not re.search(r'\d+', str(x))
    else x
)

df['salary'] = df.apply(
    lambda x: str(x['salary']+"-"+str(list(x['jobType'])[0]) )
    if not re.search(r'\d+', str(list(x['jobType'])[0])) 
    else str(x['salary']), 
    axis=1
)
professions[5:7]

df['jobType'] = df['jobType'].map(
    lambda x: '' 
    if not re.search(r'\d+', str(x))
    else x
)
"""
df.describe()

C:\Users\PC\AppData\Local\Temp\ipykernel_12440\1126720353.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: tuple(x) if isinstance(x, list) else x)


,positionTitle,companyName,location,salaryInfoAndJobType,description,datePosted,salary,jobType,shiftAndSchedule,jobBenefits,updatedDate,country,profession,language
count,21274,21274,21274,21274,21274,20921,21274,21274,21274,21274,21274,21274,21274,21274
unique,8899,5077,4827,3325,7369,61,1944,174,328,2040,13,2,14,2
top,Office Administrator,Accenture Federal Services,Paris (75),Full-time,,,,(),,(),2024-09-13,US,"Agente commerciale ou agent commercial, politi...",en
freq,255,244,591,2828,172,4448,9936,6849,17068,10114,4373,14607,4204,15545


In [15]:
for column in df.columns:
    print(column)

positionTitle
companyName
location
salaryInfoAndJobType
description
datePosted
salary
jobType
shiftAndSchedule
jobBenefits
updatedDate
country
profession
language


In [16]:
df[df['positionTitle'].str.contains('rn', case=False)==True]['positionTitle'].tolist()

['Overnight Cabin Supervisor-SFO Airport',
 'Phlebotomist Technician (Morning Shift)',
 'All-Source Intelligence Analyst Journeyman',
 'Direct Support Professional - Overnights',
 'Certified Nursing Assistant (CNA) Afternoons',
 'Instructional Designer (Digital Learning)',
 '10 hr schedule Afternoon/Midnight split Shelter Specialist',
 'Overnight Relief Crisis Intervention Specialist (QMHP)',
 'Resident Coordinator - PRN-Day and Swing Shift',
 'Patient Care RN in the Acute Department',
 'Northern & Western Montana Outreach Specialist (Havre)',
 'Veteran Care Coordinator-RN',
 'Bilingual PRN Early Childhood Instructor',
 'BRONSON BATTLE CREEK LICENSED PRACTICAL NURSE (LPN) PRN VARIABLE HOURS',
 'Horn Teaching Artist - North Side',
 '10 hr schedule Afternoon/Midnight split Shelter Specialist',
 'All-Source Intelligence Analyst Journeyman',
 'Overnight Relief Crisis Intervention Specialist (QMHP)',
 'Direct Support Professional (Part-time Sleep Overnights)',
 'Assistant Director, Graduate

In [17]:
def plot_bar_chart(df, column):
    plt.figure(figsize=(10, 6))
    for i in range(0 , df[column].nunique() , 20):
        df[column].value_counts()[i:i+20].plot(kind='bar')
        plt.show()

In [19]:
# from pandasql import sqldf

In [ ]:
plot_bar_chart(df, 'jobType')

In [ ]:
df_counts = df[['jobType','salary']].value_counts()[:20].plot(kind='bar')
# df_counts.plot(kind='scatter', x='positionTitle', y='location', s=df_counts['count'] * 10)
# plt.xticks(rotation=45)
# plt.show()
df_counts

In [ ]:
df['salary'] = df.apply(
    lambda x: "" 
    if not re.search(r'\d+', str(x['salary'])) 
    else x['salary'], 
    axis=1
)

# Afficher les 10 valeurs les plus fréquentes dans salary
df['salary'].value_counts()[:20].plot(kind='bar')

In [23]:
def transform_position_title(df):
    """
    This function cleans a DataFrame containing job posting information by following these steps:
    Steps:
    1. Remove dates:
    2. Remove salary rates:
    3. Remove hourly rates:
    4. Replace codes with actual position titles where possible:
    5. Rename bad encodings:
    6. Remove job type and shift/schedule information:
    7. Remove location if possible:
    Arguments:
    - df: DataFrame containing the job postings data to be cleaned.
    Returns:
    - A cleaned DataFrame according to the steps above.
    """

    # Remove dates
    date_pattern1 = r'\d{1,2}/\d{1,2}/\d{2,4}'
    date_pattern2 = r'\d{1,2}-\d{1,2}-\d{2,4}'
    date_pattern3 = r'\d{1,2}\.\d{1,2}\.\d{2,4}'
    date_pattern4 = r'\d{1,2}\s\w+\s\d{2,4}'
    date_pattern5 = r'\w+\s\d{1,2},\s\d{4}'
    date_pattern6 = r'\d{4}-\d{2}-\d{2}'
    date_patterns = [date_pattern1, date_pattern2, date_pattern3, date_pattern4, date_pattern5, date_pattern6]
    df = df.map(lambda x: re.sub(pattern, '', x) for pattern in date_patterns)

    # Remove month and year
    month_year_pattern = r'\b(janvier|january|février|february|mars|march|avril|april|mai|may|juin|june|juillet|july|août|august|septembre|september|octobre|october|novembre|november|décembre|december)\s\d{4}\b'
    df = df.map(lambda x: re.sub(month_year_pattern, '', x))

    # Remove salary rates
    pattern_salary_ca = re.compile(r'(?:De\s)?([\d\s]+)\s?\$(?:\s?à\s([\d\s]+)\s?\$\s?)(?:par\s|per\s|annuel\s|yearly\s)?(year|annum|an)\s*', re.IGNORECASE)
    pattern_salary_us = re.compile(r'(?:From\s)?\$\d+(?:,\d{3})*(?:\.\d{2})?(?:–\$\d+(?:,\d{3})*(?:\.\d{2})?)?\s?(?:a|per)\s(hour|day|week|month|year)\s*', re.IGNORECASE)
    pattern_salary_fr = re.compile(r'(?:De\s)?([\d\s]+)\s?€\s?à\s([\d\s]+)\s?€\s?(?:par\s|per\s|annuel\s|yearly\s|annum\s)?(an|year)\s*', re.IGNORECASE)
    salary_patterns = [pattern_salary_ca, pattern_salary_us, pattern_salary_fr]
    df = df.map(lambda x: re.sub(pattern, '', x) for pattern in salary_patterns)

    # Remove hourly rates
    time_units_pattern = r'\b(heure|hrs|hour|day|jour|week|semaine)\b'
    df = df.map(lambda x: re.sub(time_units_pattern, '', x))

    # Replace codes with actual position titles where possible(eg. RN -> Registered Nurse)
    
    # Rename bad encodings

    # Remove job type and shift/schedule information
    jobType_pattern = r'\b(temps\splein|full\s?time|part\s?time|temps\s?partiel|contract|contractuel|permanent|temporaire|temporary|contractual\b'
    shiftAndSchedule_pattern = r'\b(jour|day|soir|evening|nuit|night|weekend|fin\sde\ssemaine|weekend|rotating|rotatif|rotative|rotating\b'
    df = df.map(lambda x: re.sub(jobType_pattern, '', x))
    df = df.map(lambda x: re.sub(shiftAndSchedule_pattern, '', x))

    # Remove location if possible
    


    
    return df

def transform_salary_and_job_type(df):
    """
    This function processes salary and job type information in a DataFrame, ensuring consistency in language and data structure, following these steps:
    Steps:
    1. Extract salary information from combined fields:
    2. Handle missing salary information:
    3. Standardize hybrid data (language and format)
    4. Detect and translate language:
    Arguments:
    - df: DataFrame containing job posting data with salary, job type, and schedule information to be processed.
    Returns:
    - A cleaned DataFrame with standardized salary, job type, schedule, and translated information where necessary.
    """
    pass

def transform_company_name(df): # ===> Yousra
    """
    This function cleans a DataFrame by identifying and removing bad encodings, specifically sequences of squares or other invalid characters in text fields (e.g., names).

    Steps:
    1. Detect bad encodings:
    2. Remove or replace bad encodings:

    Arguments:
    - df: DataFrame containing text fields where bad encodings (e.g., sequences of squares) need to be removed.

    Returns:
    - A cleaned DataFrame with bad encodings removed from text fields.


    """
    pass
def transform_location(df):  # ===> Dan 
    """
    This function cleans a DataFrame by removing special characters, specifically at the beginning of text fields, and addressing bad encodings.

    Steps:
    1. Remove bad encodings:
    2. Remove special characters at the beginning of text fields:

    Arguments:
    - df: DataFrame containing text fields where bad encodings and special characters at the beginning of strings need to be removed.

    Returns:
    - A cleaned DataFrame with special characters removed from the beginnings of text fields and bad encodings corrected.
    """

In [24]:
def transfrom():
    x=transform_position_title(x)
    x=transform_salary_and_job_type(x)
    x=transform_salary(x)
    x=transform_company_name(x)
    x=transform_location(x)

In [ ]:
plot_bar_chart(df, 'positionTitle')

#### We can see that salaries are too different we can try to put them together in intervals